In [1]:
#导入使用的库
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

In [2]:
#从职位详情页面内获取职位要求
def getjobneeds(positionId):
    url = 'https://www.lagou.com/jobs/{}.html'
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
        'Host':'www.lagou.com',
        'Referer':'https://www.lagou.com/jobs/2949976.html',
        'Upgrade-Insecure-Requests':'1'
        }
    res = requests.get(url.format(positionId),headers = headers,timeout = 30)
    soup = BeautifulSoup(res.text,'html.parser')
    need =  ' '.join([p.text.strip()for p in soup.select('.job_bt div')])
    return need

In [3]:
#获取职位具体信息
def getjobdetails(jd):
    results= {}
    results['businessZones'] = jd['businessZones']
    results['companyFullName'] = jd['companyFullName']
    results['companyLabelList'] = jd['companyLabelList']
    results['companySize'] = jd['companySize']
    results['district'] = jd['district']
    results['salary'] = jd['salary']
    results['secondType'] = jd['secondType']
    results['workYear'] = jd['workYear']
    results['education'] = jd['education']
    results['firstType'] = jd['firstType']
    results['secondType'] = jd['secondType']
    results['positionName'] = jd['positionName']
    results['positionLables'] = jd['positionLables']
    results['positionAdvantage'] = jd['positionAdvantage']
    positionId = jd['positionId']
    results['need'] = getjobneeds(positionId)
    return results

In [4]:
#获取整个页面上的职位信息
def parseListLinks(url):
    jobs = []
    from_data = {'first':'true',
                'pn':'1',
                'kd':'数据分析师'}
    
    
    headers = {'Host':'www.lagou.com',
               'Referer':'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90%E5%B8%88?labelWords=&fromSearch=true&suginput=',
               'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
               'X-Anit-Forge-Code':'0',
               'X-Anit-Forge-Token':'None',
               'X-Requested-With':'XMLHttpRequest',
               'Cookie':'user_trace_token=20170911214403-8a9e01e0-a7e2-4a98-a2e1-51d2bda8cc14; LGUID=20170911214410-4a485ad6-96f7-11e7-9167-5254005c3644; _putrc=05210356AF232BFA; login=true; unick=%E8%AF%95%E8%AF%95; showExpriedIndex=1; showExpriedCompanyHome=1; showExpriedMyPublish=1; hasDeliver=24; TG-TRACK-CODE=search_code; SEARCH_ID=f78c44216900481a9d1a17d2b7a9fcb2; index_location_city=%E6%9D%AD%E5%B7%9E; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1505137450,1505189661,1505212721,1505384326; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1505386635; _gid=GA1.2.548341228.1505137448; _ga=GA1.2.828968221.1505137448; LGRID=20170914185714-775e238d-993b-11e7-9274-525400f775ce; JSESSIONID=ABAAABAACDBAAIA944F047CDEAC47D5E3D281822464B58B'
              }
    
    res = []
    for n in range(30):
        from_data['pn'] = n + 1
        res.append(requests.post(url,headers = headers,data = from_data,timeout = 30))
        
    jd = []
    for m in range(len(res)):
        jd.append(json.loads(res[m].text)['content']['positionResult']['result'])
    for j in range(len(jd)):
        for i in range(15):
            jobs.append(getjobdetails(jd[j][i]))
    time.sleep(30)
    return jobs

In [5]:
url = 'https://www.lagou.com/jobs/positionAjax.json?city=%E6%9D%AD%E5%B7%9E&needAddtionalResult=false&isSchoolJob=0'
jobs_total = parseListLinks(url)

In [6]:
jobs_total

[{'businessZones': ['仓前'],
  'companyFullName': '杭州橙鹰数据技术有限公司',
  'companyLabelList': ['年底双薪', '带薪年假', '午餐补助', '定期体检'],
  'companySize': '150-500人',
  'district': '余杭区',
  'education': '本科',
  'firstType': '开发/测试/运维类',
  'need': '职位描述：\n1.深入理解创新业务的方向和战略，进行相关业务数据的采集、处理和开发；\n2.负责公司业务创新领域的数据拓展分析，提供数据决策支持；\n3.运用各种数据分析方法，建立业务指标监控体系，发现业务问题，并给出行动建议，以推动业务成长；\n4.深入理解公共安全业务，从用户的角度思考问题，通过数据分析，找出业务价值点，推动建立数据化服务；\n5.针对具体的业务问题，规划、设计基于数据挖掘的解决方案。\n\n职位要求：\n1.本科及以上学历，数学、统计学、计算机等工科专业优先；\n2.五年以上数据分析经验，大量数据处理经验，一定的大规模复杂数据处理经验，对跨业务领域的结构化、非架构化数据分析方法有一定见解，对用户画像、行为分析、时间序列分析等有深刻见解；\n3.对数据有好奇心，有较好的sense，根据业务特点进行数据抽象和建模，且能深究；\n4.具有较强的逻辑思维能力、数据洞察能力和沟通协调能力，能够独立完成分析项目；\n5.良好的业务推动、跨部门协作能力和全局观；\n6.为人正直，正能量，乐于面对挑战，思维活跃，善于创新。',
  'positionAdvantage': '阿里巴巴,大数据,蓝海,牛人多',
  'positionLables': ['信息安全', '大数据', '数据挖掘'],
  'positionName': '资深数据分析师',
  'salary': '15k-30k',
  'secondType': '数据开发',
  'workYear': '5-10年'},
 {'businessZones': ['高新文教区', '古荡', '西湖'],
  'companyFullName': '杭州财米科技有限公司',
  'companyLabelList': ['年底多薪', '

In [6]:
df = pd.DataFrame(jobs_total)
df.head

<bound method NDFrame.head of           businessZones    companyFullName            companyLabelList  \
0                  [仓前]       杭州橙鹰数据技术有限公司    [年底双薪, 带薪年假, 午餐补助, 定期体检]   
1       [高新文教区, 古荡, 西湖]         杭州财米科技有限公司    [年底多薪, 岗位晋升, 定期体检, 五险一金]   
2                  [宁围]         传化物流集团有限公司   [行业爆炸, 扁平管理, “家”文化, 牛人团队]   
3      [文三路, 古荡, 高新文教区]      杭州铜板街网络科技有限公司    [创新开放, 团队牛X, 全员期权, 高速成长]   
4         [小河, 拱宸桥, 上塘]    杭州铭师堂教育科技发展有限公司    [自主盈利, 扁平管理, 技术大牛, 岗位晋升]   
5                  None       浙江执御信息技术有限公司    [五险一金, 带薪年假, 节日礼物, 年度旅游]   
6       [西湖, 古荡, 高新文教区]      杭州微拍堂文化创意有限公司                [岗位晋升, 年度旅游]   
7                  None  浙江多多互联网金融信息服务有限公司    [节日礼物, 年底双薪, 股票期权, 带薪年假]   
8                  None    杭州钱塘大数据交易中心有限公司     [定期体检, 专项奖金, 带薪年假, 生日趴]   
9          [小河, 大关, 和睦]  杭州投融谱华互联网金融服务有限公司    [午餐补助, 定期体检, 带薪年假, 年度旅游]   
10                 [西溪]       杭州网营科技股份有限公司    [五险一金, 周末双休, 午餐补助, 晚餐补助]   
11                 None         杭州优行科技有限公司    [优惠购车, 定期调薪, 广阔平台, 福利丰厚]   
12      

In [7]:
df.to_excel('拉勾网爬虫职位信息.xlsx')